In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
xl = pd.ExcelFile("./NWM_data_updated.xlsx", engine='openpyxl')
a=xl.sheet_names

#loading the data set
dataframe_collection = {}
i=0
for i in range(33):
    if i==0 :
        dataframe_collection[i] = xl.parse(a[i],converters={"STAID_1":str})
    else:
        dataframe_collection[i] = xl.parse(a[i])


obsflow = dataframe_collection[2]
station_data_total = dataframe_collection[0]

In [2]:
for v in range(0,30):
    modelflow= dataframe_collection[v+3]
    for b in range(0,5):
            

            station_data = station_data_total.sample(frac=0.9)
            station_data_ungagued = station_data_total.loc[~station_data_total.index.isin(station_data.index)]

            station_data = station_data.reset_index(drop=True)
            station_data_ungagued = station_data_ungagued.reset_index(drop=True)
            k=0
            column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
            df = pd.DataFrame(columns = column_names)  
            for index, row in station_data.iterrows():
                print(index)
                sta_id = station_data.iloc[index,0]
                dis2sea=station_data.iloc[index,1]
                land_use=station_data.iloc[index,2]
                soil_texture1=station_data.iloc[index,3]
                soil_texture2=station_data.iloc[index,4]
                soil_texture3=station_data.iloc[index,5]
                soil_texture4=station_data.iloc[index,6]
                soil_texture5=station_data.iloc[index,7]
                soil_texture6=station_data.iloc[index,8]
                soil_texture7=station_data.iloc[index,9]
                soil_texture8=station_data.iloc[index,10]
                soil_texture9=station_data.iloc[index,11]
                soil_texture10=station_data.iloc[index,12]
                topoindex = station_data.iloc[index,13]
                drainagearea =  station_data.iloc[index,14]
                obs = obsflow[sta_id]/drainagearea
                mod = modelflow[sta_id]/drainagearea

                for i in range(979):
                    df.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
                    k=k+1
            df =df.dropna()



            #train_test Split

            import numpy as np
            from sklearn.model_selection import train_test_split
            df=df.sample(frac=1).reset_index(drop=True)
            df.iloc[:,15]= abs(df.iloc[:,15] - df.iloc[:,14])

            x, y = df.iloc[:,0:15], df.iloc[:,15]
            y = y.values.reshape(y.shape[0],1)
            x_train, x_test, y_train, y_test = \
                train_test_split(x, y, test_size=0.1, 
                                             random_state=0)

            #standardize
            x_mean = x_train.mean(axis=0)
            x_train -= x_mean
            x_std = x_train.std(axis=0)
            x_train /= x_std

            x_test -= x_mean
            x_test /= x_std 

            y_mean = y_train.mean(axis=0)
            y_train -=y_mean
            y_std=y_train.std(axis=0)
            y_train /=y_std

            y_test -=y_mean
            y_test /=y_std


            #model Train
            from keras.models import Sequential
            from keras.layers import Dense
            from keras import layers
            from keras.layers import LSTM
            from keras import backend as K 


            model= Sequential()

            model.add(layers.Dense(512, activation='relu',input_shape=(x_train.shape[1],)))
            model.add(layers.Dense(256, activation='relu'))
            model.add(layers.Dense(64, activation='relu'))
            model.add(layers.Dense(32, activation='relu'))
            model.add(layers.Dense(16, activation='relu'))
           # model.add(layers.Drut(opo0.4))
        #model.add(layers.Dense(4, activation='relu'))
            model.add(layers.Dense(1))
            model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
            #end edit    
            # Fitting the RNN to the Training set
            history=model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 1024, epochs = 13)
            #istory=model.fit(x_train, y_train, batch_size = 1000, epochs = 20) 3
            model.save('./spatial_DLmodels/my_model_%s_day_%s_iter.h5'%(v,b))


            #Preparing Second dataset


            k=0
            column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
            df2 = pd.DataFrame(columns = column_names)  
            for index, row in station_data_ungagued.iterrows():
                print(index)
                sta_id = station_data_ungagued.iloc[index,0]
                dis2sea=station_data_ungagued.iloc[index,1]
                land_use=station_data_ungagued.iloc[index,2]
                soil_texture1=station_data_ungagued.iloc[index,3]
                soil_texture2=station_data_ungagued.iloc[index,4]
                soil_texture3=station_data_ungagued.iloc[index,5]
                soil_texture4=station_data_ungagued.iloc[index,6]
                soil_texture5=station_data_ungagued.iloc[index,7]
                soil_texture6=station_data_ungagued.iloc[index,8]
                soil_texture7=station_data_ungagued.iloc[index,9]
                soil_texture8=station_data_ungagued.iloc[index,10]
                soil_texture9=station_data_ungagued.iloc[index,11]
                soil_texture10=station_data_ungagued.iloc[index,12]
                topoindex = station_data_ungagued.iloc[index,13]
                drainagearea =  station_data_ungagued.iloc[index,14]
                obs = obsflow[sta_id]/drainagearea
                mod = modelflow[sta_id]/drainagearea

                for i in range(979):
                    df2.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
                    k=k+1
            df2 =df2.dropna()

            x, y = df2.iloc[:, 0:15], df2.iloc[:,15]
            x-= x_mean
            x /= x_std



            from keras.models import load_model
            model = load_model('./spatial_DLmodels/my_model_%s_day_%s_iter.h5'%(v,b))
            model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

            z=model.predict(x)

            x= x*x_std
            x=x+x_mean
            d_area_x = x.iloc[:,13]
            x.iloc[:,14]=x.iloc[:,14]*d_area_x

            y=y*d_area_x

            z=z*y_std
            z=z+y_mean
            d_area_x=np.array(d_area_x)
            d_area_x.shape = [d_area_x.shape[0],1]
            z=z*d_area_x

            d_x= pd.DataFrame(x.iloc[:,14])
            d_y=pd.DataFrame(y)
            d_z=pd.DataFrame(z)

            d_xy=pd.concat([d_x, d_y,d_z], axis=1)
            d_xy.columns = ['Modelflow', 'Obsflow','Error']
            d_xy

            d_xy.to_csv("./graph_data/reviewer_Eror_WholeData_abs_iter_%s_lead_%s_iter.csv"%(v,b))




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2022-09-07 03:48:01.695733: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-07 03:48:01.695795: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (crhtc53): /proc/driver/nvidia/version does not exist
2022-09-07 03:48:01.697152: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/15
212/212 [==============================] - 7s 24ms/step - loss: 0.7249 - mae: 0.1752 - val_loss: 0.6076 - val_mae: 0.2245
Epoch 2/15
212/212 [==============================] - 4s 17ms/step - loss: 0.7014 - mae: 0.1654 - val_loss: 0.5979 - val_mae: 0.1527
Epoch 3/15
212/212 [==============================] - 4s 20ms/step - loss: 0.6908 - mae: 0.1619 - val_loss: 0.5911 - val_mae: 0.1718
Epoch 4/15
212/212 [==============================] - 7s 34ms/step - loss: 0.6891 - mae: 0.1602 - val_loss: 0.5826 - val_mae: 0.1699
Epoch 5/15
212/212 [==============================] - 8s 38ms/step - loss: 0.6878 - mae: 0.1600 - val_loss: 0.5892 - val_mae: 0.1540
Epoch 6/15
212/212 [==============================] - 9s 43ms/step - loss: 0.6890 - mae: 0.1588 - val_loss: 0.5783 - val_mae: 0.1526
Epoch 7/15
212/212 [==============================] - 9s 40ms/step - loss: 0.6877 - mae: 0.1578 - val_loss: 0.5854 - val_mae: 0.1912
Epoch 8/15
212/212 [==============================] - 6s 26ms/step - 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

